In [ ]:
from numba import njit

from random import choice
import dill
import traceback
import logging

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

### Task_1

Попробуйте улучшить работу нейронной сети(разобранную на уроке) обучавшейся на датасет Fashion-MNIST. Опишите в комментарии к уроку - какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

##### Functions:

In [ ]:
def build_model(layers_number, 
                activation,
                neurons_in_layer):
    layers = [keras.layers.Flatten(input_shape=(28, 28))]
    for i in range(layers_number):
        layers.append(keras.layers.Dense(neurons_in_layer, activation=activation))
    layers.append(keras.layers.Dense(10))
    model = keras.Sequential(layers)
    return model

def learn(model, 
          optimizer,
          metrics: list,
          epochs: int):
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=metrics)

    model.fit(train_images, train_labels, epochs=epochs,  verbose=1)
    
def evaluate(trained_model):
    test_loss, test_acc = trained_model.evaluate(test_images,  test_labels, verbose=2)
    return round(test_acc, 4)

def learn_evaluate_many_times(model, 
                              optimizer,
                              metrics: list,
                              epochs: int,
                              runs=5):
    accuracies = []
    weights_path = 'weights.weights'
    model.save_weights(weights_path)
    
    for i in range(runs):
        model.load_weights(weights_path)
        learn(model=model, 
              optimizer=optimizer,
              metrics=metrics,
              epochs=epochs)
        accuracies.append(evaluate(model))
    accuracies = np.array(accuracies)
    return (accuracies.mean(), round(accuracies.std(), 4), accuracies)

##### Baseline:

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

results = learn_evaluate_many_times(model=model, 
                          optimizer='adam',
                          metrics=['accuracy'],
                          epochs=5,
                          runs=5)

In [ ]:
results

Baseline metrics (mean accuracy, accuracy std):  (0.8704, 0.0049)

##### Research:

Рандомлю комбинации значений параметров и оцениваю на отложенной выборке результаты (по метрике среднее значение accuracy за 5 прогонов и std этой метрики за эти 5 прогонов).

In [ ]:
epochs = np.arange(5, 15, 1)
epochs_ = []

neurons_numbers = np.arange(1, 325, 5)
neurons_numbers_ = []

layers_numbers = np.arange(1, 5, 1)
layers_numbers_ = []

activations = ['relu', 'softmax', 'sigmoid']
activations_ = []

results_ = []


for i in range(25):
    try:
        epoch = choice(epochs)
        neurons_number = choice(neurons_numbers)
        layers_number = choice(layers_numbers)
        activation = choice(activations)

        model = build_model(layers_number=layers_number, 
                    activation=activation,
                    neurons_in_layer=neurons_number)

        r = learn_evaluate_many_times(model=model, 
                              optimizer='adam',
                              metrics=['accuracy'],
                              epochs=epoch,
                              runs=5)
        epochs_.append(epoch)
        neurons_numbers_.append(neurons_number)
        layers_numbers_.append(layers_number)
        activations_.append(activation)

        results_.append(r)
        print('RESULTS:', {'epoch': epoch,
               'neurons_number': neurons_number,
               'layers_number': layers_number,
               'activation': activation, 
               'mean_accuracy': r[0],
               'accuracy_std': r[1]})
    except:
        print(traceback.format_exc())

In [ ]:
results = pd.DataFrame({'epochs': epochs_,
                       'neurons_number': neurons_numbers_,
                       'layers_number': layers_numbers_,
                       'activation': activations_,
                       'mean_accuracy': [val[0] for val in results_],
                       'accuracy_std': [val[1] for val in results_]})

results.sort_values(by='mean_accuracy', ascending=False)

In [ ]:
plt.hist(results['mean_accuracy'])

In [ ]:
# Влияние параметров.
def param_impact(param, results):
    less = results.loc[results[param] < results[param].mean(), 'mean_accuracy'].mean()
    more = results.loc[results[param] > results[param].mean(), 'mean_accuracy'].mean()
    
    return round(less, 4), round(more, 4), round((more - less) / less, 4)

In [ ]:
results.info()

In [ ]:
print('epochs', param_impact('epochs', results))
print('neurons_number', param_impact('neurons_number', results))
print('layers_number', param_impact('layers_number', results))

Результаты выглядят не соответствующими стандартным закоромерностям, что связано, вероятно, с малым кол-вом прогонов и смещенными выборками по тем признакам, которые не участвовали в разделении при оценке влияния параметра.

In [ ]:
results.groupby('activation').mean()

Какими бы нерепрезентативными не были выборки, но, похоже, softmax никак не годится для внутренних слоев.

In [ ]:
results.sort_values(by='mean_accuracy', ascending=False).head(1)

##### Выводы:

Baseline metrics (mean accuracy, accuracy std): (0.8704, 0.0049)
best learn metrics (mean accuracy, accuracy std): (0.8886, 0.0006)

Точность выросла почти на 2%, и существенно упал разброс модели.

Росту качества модели способствовали факторы:
- увеличение числа эпох обучения,
- увеличение числа нейронов в слое.

### Task_2:

Поработайте с документацией TensorFlow 2. Попробуйте найти полезные команды TensorFlow,  не разобранные на уроке.

### Task_3

Попробуйте обучить нейронную сеть на TensorFlow 2 на датасете imdb_reviews

##### EDA:

Получаем данные:

In [59]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data()

In [60]:
type(train_data), type(train_labels)

(numpy.ndarray, numpy.ndarray)

In [61]:
type(train_data)

numpy.ndarray

In [62]:
train_data.shape, train_labels.shape

((25000,), (25000,))

In [63]:
test_data.shape, test_labels.shape

((25000,), (25000,))

In [64]:
type(train_data[0]), len(train_data[0])

(list, 218)

In [65]:
type(train_data[1]), len(train_data[1])

(list, 189)

In [66]:
type(train_data[7]), len(train_data[7])

(list, 562)

In [67]:
train_labels[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int64)

Дата-сет - массив списков разной длины.

In [68]:
word_index = imdb.get_word_index()

Смотрю среднюю длину (кол-во слов) одного текста.

In [69]:
%%time

def calc_mean_len(arr):
    lengths = []
    for a in arr:
        lengths.append(len(a))
    lengths = np.array(lengths)
    print(lengths.mean())
        
calc_mean_len(train_data)

238.71364
Wall time: 12 ms


In [70]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=0, padding='post', maxlen=238)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=0, padding='post', maxlen=238)

In [71]:
len(train_data[0]), len(train_data[1]), len(train_data[7])

(238, 238, 238)

##### Learn model:

In [72]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='relu', input_shape=(238,)),
    keras.layers.Dense(5, activation='relu'),
#     keras.layers.Dense(256, activation='relu'),
#     keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2)
])

model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

model.fit(train_data, train_labels, epochs=1,  verbose=1)

train_loss, train_acc = model.evaluate(train_data,  train_labels, verbose=1)
print('train_accuracy', round(train_acc, 4))

test_loss, tes_acc = model.evaluate(test_data,  test_labels, verbose=1)
print('test_accuracy', round(test_acc, 4))

Train on 25000 samples
25000/25000 [==============================] - 2s 60us/sample - loss: 0.7074 - accuracy: 0.5009
train_accuracy 0.5009
25000/25000 [==============================] - 1s 53us/sample - loss: 1.0198 - accuracy: 0.4992
test_accuracy 0.5149


##### Выводы:

Как-то подозрительно стабильно получается результат 0,5149 на отложенной выборке. И ничем его оттуда не сдвинуть, похоже, где-то ошибка, интересно где?